# Iteracyjna wspinaczka

In [1]:
from tracemalloc import start
import pandas as pd
import xlrd
import random
import copy

In [2]:
df48 = pd.read_excel("Dane_TSP_48.xlsx", index_col=0)
df76 = pd.read_excel("Dane_TSP_76.xlsx", index_col=0)
df127 = pd.read_excel("Dane_TSP_127.xlsx", index_col=0)

In [3]:
def droga(trasa,df):
    suma=0
    for i in range(0,len(trasa)-1):
        suma = suma + df[trasa[i]].iloc[trasa[i+1]-1]
    return suma

In [4]:
def swapPositions(list, pos1, pos2):
    list[pos1], list[pos2] = list[pos2], list[pos1]
    return list

In [5]:
def invertPositions(list, pos1, pos2):
    list = swapPositions(list, pos1, pos2)  

    small_list = list[pos1+1:pos2]  
    reversed_list = small_list[::-1]

    for i in range(0, len(small_list)):
        list[pos1+i+1] = reversed_list[i]

    return list

In [7]:
def insertPositions(list, pos1, pos2):
    if pos2 < pos1:
        temp = pos1
        pos1 = pos2
        pos2 = temp

    small_list = list[pos1+1:pos2]  

    list[pos1+1] = list[pos2]  

    for i in range(0, len(small_list)):
        list[pos1+i+2] = small_list[i]

    return list

In [22]:
#wspinaczka(dane, liczba iteracji bez powtorzen z rzedu, liczba startów(multistart), sasiad(1-swap,2-inwersja,3-insercja), liczba iteracji)
def wspinaczka(df, iteracjebezpoprawy,liczbaprob,sasiad,iteracje):
    najkrotsza = list(range(1, len(df)+1))
    najkrotszaodleglosc = 1000000
    for i in range(liczbaprob):
        trasa = list(range(1, len(df)+1))
        random.shuffle(trasa)
        odleglosc = droga(trasa,df)
        print("Trasa początkowa: ", trasa)
        print("Początkowa odległość: ", odleglosc)
        liczbaiteracji = 0
        ibez = 0
        ilemiast=len(df)
        while iteracjebezpoprawy!=ibez and liczbaiteracji!=iteracje:
            nowatrasa = copy.deepcopy(trasa)
            a=random.randint(0,ilemiast-1)
            b=random.randint(0,ilemiast-1)
            while a==b:
                b=random.randint(1,ilemiast-1)
            if sasiad==1:
                swapPositions(nowatrasa,a,b)
            elif sasiad==2:
                invertPositions(nowatrasa,a,b)
            elif sasiad==3:
                insertPositions(nowatrasa,a,b)
            nowaodleglosc = droga(nowatrasa,df)
            if nowaodleglosc >= odleglosc:
                ibez = ibez+1
            else: 
                trasa=nowatrasa
                odleglosc=nowaodleglosc
                ibez=0
            liczbaiteracji=liczbaiteracji+1
        if najkrotszaodleglosc > odleglosc:
            najkrotsza = trasa
            najkrotszaodleglosc = odleglosc
            iteracjewnajkrotszej = liczbaiteracji

    print("Liczba wykonanych iteracji: ", iteracjewnajkrotszej)
    print("Najlepsza trasa: ",najkrotsza)
    print("Odleglość najlepszej trasy: ",najkrotszaodleglosc)

Przykładowe wywołanie algorytmu:

In [ ]:
wspinaczka(df48,500,10,1,2500)

# Symulowane wyrzażanie

In [8]:
import pandas as pd
import random
import copy
import math

In [11]:
#funkcja służąca do zmiany indeksów wczytywanych plików csv
def change_index(dane):
    dane.columns.values[0] = 0
    dane = dane.set_index(0)
    return dane

In [10]:
#funkcja służąca do wylosowania początkowego rozwiązania
def random_shuffle(len_dane):
    listaMiast = list(range(1, len_dane+1))
    poczatkoweRozwiazanie = listaMiast.copy()
    random.shuffle(poczatkoweRozwiazanie)
    return poczatkoweRozwiazanie

In [12]:
def algorytm(dane, alfa, t, liczbaProbWEpoce, tMin):
    lista = dane.values.tolist()
    poczatkoweRozwiazanie = random_shuffle(len(lista))
    epoka = 1
    suma = 499999
    sumaSuma = 500000 #zeby mozna było zapisać najmniejszą sume, która przewinęła się podczas dzialania algorytmu
    sumaLista = []
    while t > tMin:
        proba = 0
        while proba < liczbaProbWEpoce:
            odleglosciNowe = []
            odleglosciStare = []
            noweRozwiazanie = copy.deepcopy(poczatkoweRozwiazanie)
            dwieLosowe = random.sample(range(len(lista)), 2)
            
            #wyznaczanie rozwiązania sąsiedniego
            #Rodzaj sąsiedztwa - SWAP
            noweRozwiazanie[dwieLosowe[0]] = poczatkoweRozwiazanie[dwieLosowe[1]]
            noweRozwiazanie[dwieLosowe[1]] = poczatkoweRozwiazanie[dwieLosowe[0]]
            
            """#Rodzaj sąsiedztwa - INVERSION
            if(dwieLosowe[0] < dwieLosowe[1]):
                noweRozwiazanie[dwieLosowe[0] : dwieLosowe[1] + 1] = reversed(noweRozwiazanie[dwieLosowe[0] : dwieLosowe[1] + 1])
            else:
                noweRozwiazanie[dwieLosowe[1] : dwieLosowe[0] + 1] = reversed(noweRozwiazanie[dwieLosowe[1] : dwieLosowe[0] + 1])"""
            
            """#Rodzaj sąsiedztwa - INSERTION
            if(dwieLosowe[0] < dwieLosowe[1]):
                noweRozwiazanie.insert(dwieLosowe[0]+1, noweRozwiazanie.pop(dwieLosowe[1]))
            else:
                noweRozwiazanie.insert(dwieLosowe[1]+1, noweRozwiazanie.pop(dwieLosowe[0]))"""
            
            #dodawanie do pustej listy odległości międzi miastami
            for i in range(0, len(lista)-1):
                odleglosciNowe.append(lista[noweRozwiazanie[i]-1][noweRozwiazanie[i+1]-1])
                odleglosciStare.append(lista[poczatkoweRozwiazanie[i]-1][poczatkoweRozwiazanie[i+1]-1])

            #dodawanie do listy odległości międzi ostatnim a pierwszym miastem
            odleglosciNowe.append(lista[noweRozwiazanie[0]-1][noweRozwiazanie[-1]-1])
            odleglosciStare.append(lista[poczatkoweRozwiazanie[0]-1][poczatkoweRozwiazanie[-1]-1])

            nowaSuma = sum(odleglosciNowe)
            staraSuma = sum(odleglosciStare)

            #sprawdzanie czy nowe rozwiązanie jest lepsze od aktualnego
            if(nowaSuma < staraSuma):
                poczatkoweRozwiazanie = noweRozwiazanie
                suma = nowaSuma
            else:
                if(random.uniform(0, 1) < math.exp(-(nowaSuma - staraSuma)/t)):
                    poczatkoweRozwiazanie = noweRozwiazanie
                    suma = nowaSuma
            proba += 1
        t = t * alfa
        #sprawdzanie czy nowa najmniejsza suma jest mniejsza od tej do tej pory najmniejszej
        if(sumaSuma > suma):
            sumaSuma = suma
            sumaLista = noweRozwiazanie
    print(poczatkoweRozwiazanie)
    print("Suma", suma)
    print("Możliwa niższa suma", sumaSuma)
    if(sumaSuma < suma):
        print("Kolejność miast:", sumaLista)

In [17]:
data48 = pd.read_excel('Dane_TSP_48.xlsx')
dane48 = change_index(data48)

data76 = pd.read_excel('Dane_TSP_76.xlsx')
dane76 = change_index(data76)

data127 = pd.read_excel('Dane_TSP_127.xlsx')
dane127 = change_index(data127)

Przykładowe wywołanie algorytmu:

In [ ]:
dane = [dane48, dane76, dane127]

print("++++++++ 22 ++++++++")
for i in range(0,len(dane)):
    print("======== Zestaw:", i+1, "========")
    n = 0
    while n < 4:
        print("Powtórzenie:", n+1)    
        algorytm(dane[i], 0.99, 6000, 30000, 10)
        n += 1

# Tabu Search

In [13]:
import pandas as p
import random
import itertools as it
import copy
import winsound
from datetime import datetime

In [14]:
globalDiff = None
globalLenData = 0

In [15]:
# preparing data
def change_index(data):
    data = data.rename(columns={"": 0})
    data.columns.values[0] = 0
    data = data.set_index(0)
    return data


# excel's method Index
def index(data, simpleData, numbers):
    global globalDiff, globalLenData
    for i in range(0, globalLenData-1):
        globalDiff[i] = simpleData[numbers[i]-1][numbers[i+1]-1]  # searching diff for neighbour cities
    globalDiff[globalLenData-1] = simpleData[numbers[0]-1][numbers[globalLenData-1]-1]  # counts last city with the first
    beginSum = sum(globalDiff)
    return beginSum

# generate list of random numbers from in the range
def random_shuffle(len_data):
    listOfNumbers = list(range(1, len_data+1))
    shuffledNum = listOfNumbers.copy()
    random.shuffle(shuffledNum)
    return shuffledNum


def swapPositions(list, pos1, pos2):
    list[pos1], list[pos2] = list[pos2], list[pos1]
    return list


def invertPositions(list, pos1, pos2):
    list = swapPositions(list, pos1, pos2)  # ends

    small_list = list[pos1+1:pos2]  # interval
    reversed_list = small_list[::-1]

    for i in range(0, len(small_list)):
        list[pos1+i+1] = reversed_list[i]

    return list


def insertPositions(list, pos1, pos2):
    if pos2 < pos1:
        temp = pos1
        pos1 = pos2
        pos2 = temp

    small_list = list[pos1+1:pos2]  # interval

    list[pos1+1] = list[pos2]  # ends

    for i in range(0, len(small_list)):
        list[pos1+i+2] = small_list[i]

    return list

# counts profit of switching places of two cities
def bestCombination(data, simpleData, listOfNumbers, combi, func):
    profit = []

    for i in range(0, len(combi)-1):
        swapList = listOfNumbers.copy()
        swapList = func(swapList, combi[i][0]-1, combi[i][1]-1)
        swapSum = index(data, simpleData, swapList)
        profit.append(swapSum)

    sortedProfit = sorted(profit)
    return sortedProfit, profit

In [16]:
def algorithm(data, maxTabuSize, maxIterNoImpro, maxIter, func):
    simpleData = data.values.tolist()
    bestList = []
    tabuList = []
    i = 0
    iNoImpro = 0
    bestSumList = []

    global globalDiff, globalLenData
    globalDiff = [None] * len(data)
    globalLenData = len(data)


    shuffledNum = random_shuffle(len(data))
    combi = list(it.combinations(shuffledNum, 2))


    while i < maxIter:
    #while iNoImpro < maxIterNoImpro:
        sortedProfit, profit = bestCombination(data, simpleData, shuffledNum, combi, func)

        for j in range(0, len(sortedProfit)-1):
            candidateSum = sortedProfit[j]
            candidate = combi[profit.index(candidateSum)]

            if i == 0:  # set bestSum for first iteration
                bestSum = sortedProfit[0]

            if tabuList.__contains__(candidate):
                continue
            else:
                shuffledNum = func(shuffledNum, candidate[0] - 1, candidate[1] - 1)
                tabuList.append(candidate)

                if candidateSum <= bestSum:
                    bestSum = candidateSum
                    bestList = copy.deepcopy(shuffledNum)

                if len(tabuList) > maxTabuSize:
                    tabuList.pop(0)

            bestSumList.append(bestSum)
            break

        if i != 0:
            if bestSumList[i] == bestSumList[i-1]:
                iNoImpro += 1
            else:
                iNoImpro = 0

        i += 1
    return bestSum, bestList, i, bestSumList

In [19]:
# read data
data48 = p.read_excel('Dane_TSP_48.xlsx')
data76 = p.read_excel('Dane_TSP_76.xlsx')
data127 = p.read_excel('Dane_TSP_127.xlsx')


data48 = change_index(data48)
data76 = change_index(data76)
data127 = change_index(data127)

data = [data48, data76, data127]

Przykładowe wywołanie algorytmu:

In [ ]:

for d in data:
    print("data")
    print(datetime.now().strftime("%H:%M:%S"))
    s = 9999999999999999999
    for i in range(4):
        bestSum, bestList, iteration, bestSumList = algorithm(data=d, maxTabuSize=150, maxIterNoImpro=0, maxIter=10000,
                                                              func=insertPositions)
        if bestSum < s:
            s = bestSum
            l = bestList
    print("bestSum:", s)
    print("bestList:", l)

winsound.Beep(500, 1000)

# Algorytm genetyczny

In [25]:
import pandas as pd
import numpy as np
import random
import operator

In [26]:
# preparing data
def change_index(data):
    data.columns.values[0] = 0
    data = data.set_index(0)
    return data

def get_list_of_cities(data):
    cities = []
    for i in range(len(data[0])):
        cities.append(i + 1)
    return cities

#Inicjalizacja populacji
def initial_population(popSize, cities):
    population = []
    for i in range(0,popSize):
        population.append(random.sample(cities, len(cities)))
    return population

#Funkcja dopasowania
def fitness_rank(population, data):
    fitnessResults = {}
    for i in range(len(population)):
        sum = 0
        for j in range(len(population[0]) - 1):
            sum += data[population[i][j] - 1][population[i][j + 1] - 1]
        fitnessResults[i] = sum
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse=False)

#Selekcja

#Wybór koła ruletki
def selection_of_roulette(ranked_population, elite_size):
    selectionResults = []
    list = np.array(ranked_population).tolist()
    sum = 0
    prop = []
    index = []
    for i in range(len(list)):
        index.append(int(list[i][0]))
    for i in range(len(list)):
        list[i][1] = list[len(list) - 1][1] - list[i][1]
        sum += list[i][1]
    for i in range(len(list)):
        if sum != 0:
            prop.append(list[i][1]/sum)
        if sum == 0:
            prop.append(1/len(list))
    for i in range(0, elite_size):
        selectionResults.append(ranked_population[i][0])
    for i in range(0, len(ranked_population)//2 - elite_size):
        selectionResults.append(np.random.choice(index, p = prop))
    return selectionResults

#Turniej
def tournament_selection(ranked_population, elite_size, k):
    selectionResults = []
    list = np.array(ranked_population).tolist()
    sum = 0
    prop = []
    for i in range(len(list)):
        list[i][1] = list[len(list) - 1][1] - list[i][1]
        sum += list[i][1]
    for i in range(len(list)):
        if sum != 0:
            prop.append(list[i][1] / sum)
        if sum == 0:
            prop.append(1 / len(list))
    for i in range(0, elite_size):
        selectionResults.append(ranked_population[i][0])
    # first random selection
    for i in range(len(ranked_population)//2 - elite_size):
        selection_x = random.randint(0,len(ranked_population) - 1)
        for j in range(k-1):
            selection_y = random.randint(0,len(ranked_population) - 1)
            if prop[selection_y] > prop[selection_x]:
                selection_x = selection_y
        selectionResults.append(ranked_population[selection_x][0])
    return selectionResults

def get_selected(population,selectionResults):
    individuals = []
    for i in range(len(selectionResults)):
        individuals.append(population[selectionResults[i]])
    return individuals

#Parowanie

def fittest_pairing(individuals):
    parents = []
    for i in range(len(individuals)//2):
        parents.append([individuals[i], individuals[i+1]])
    return parents

def random_pairing(individuals):
    parents = []
    for i in range(len(individuals)//2):
        parents.append([individuals[random.randint(0, (len(individuals) - 1))], individuals[random.randint(0, (len(individuals) - 1))]])
        while parents[i][0] == parents[i][1]:
            parents[i][1] = individuals[random.randint(0, (len(individuals) - 1))]
    return parents

#Cross-over

def cross_over_single_point(parents):
    children = []
    for i in range(len(parents)):
        index = int(random.random() * (len(parents[0][0])-1))
        childA = []
        childB = []
        for j in range(index):
            childA.append(parents[i][0][j])
            childB.append(parents[i][1][j])
        for j in parents[i][1]:
            if j not in childA:
                childA.append(j)
        for j in parents[i][0]:
            if j not in childB:
                childB.append(j)
        children.append(childA)
        children.append(childB)
    return children


def cross_over_two_point(parents):
    children = []
    for i in range(len(parents)):
        indexA = int(random.random() * len(parents[0][0]) - 1)
        indexB = int(random.random() * len(parents[0][0]) - 1)
        startIndex = min(indexA, indexB)
        endIndex = max(indexA, indexB)
        childA = []
        childB = []
        list_parentA = []
        list_parentB = []
        for j in range(startIndex):
            childA.append(parents[i][0][j])
            childB.append(parents[i][1][j])
            list_parentA.append(parents[i][0][j])
            list_parentB.append(parents[i][1][j])
        for j in range(endIndex, len(parents[0][0]), 1):
            list_parentA.append(parents[i][0][j])
            list_parentB.append(parents[i][1][j])
        for j in range(startIndex, endIndex, 1):
            for k in parents[i][1]:
                if k not in list_parentA:
                    childA.append(k)
                    list_parentA.append(k)
        for j in range(startIndex, endIndex, 1):
            for k in parents[i][0]:
                if k not in list_parentB:
                    childB.append(k)
                    list_parentB.append(k)
                    break
        for j in range(endIndex, len(parents[0][0]), 1):
            childA.append(parents[i][0][j])
            childB.append(parents[i][1][j])
        children.append(childA)
        children.append(childB)
    return children

#mutation
# Jeśli występuje elityzm, to te osobniki nie wchodzą do mutacji

def mutation(individuals, children, elite_size, prob_of_mutation = 0.001):
    mutated = []
    for i in range(elite_size, len(individuals), 1):
        mutated.append(individuals[i])
    mutated += children
    for i in range(len(mutated)):
        for swapA in range(len(mutated[0])):
            if random.random() < prob_of_mutation:
                swapB = int(random.random() * len(mutated[0]))

                cityA = mutated[i][swapA]
                cityB = mutated[i][swapB]

                mutated[i][swapA] = cityB
                mutated[i][swapB] = cityA
    return mutated

def new_population(individuals, mutated, elite_size):
    population = []
    for i in range(elite_size):
        population.append(individuals[i])
    population += mutated
    return population

In [27]:
def genetic_algorithm(data,popSize, number_of_iteration,selection_method, elite_size, pairing_method, cross_over_method, prop_of_mutation,k = 100):
    best_in_population = []
    cities = get_list_of_cities(data)
    population = initial_population(popSize,cities)
    for i in range(number_of_iteration):
        rank = fitness_rank(population,data)
        best_in_population.append([rank[0][1], population[rank[0][0]]])
        if i % 100 == 0:
            print(i)
            print(sorted(best_in_population)[0])
        if selection_method == 'tournament':
            selectionResults = tournament_selection(rank,elite_size,k)
        if selection_method == 'roulette':
            selectionResults = selection_of_roulette(rank,elite_size)
        individuals = get_selected(population,selectionResults)
        if pairing_method == 'fittest':
            parents = fittest_pairing(individuals)
        if pairing_method == 'random':
            parents = random_pairing(individuals)
        if cross_over_method == 'single_point':
            children = cross_over_single_point(parents)
        if cross_over_method == 'two_point':
            children = cross_over_two_point(parents)
        mutated = mutation(individuals,children,elite_size,prop_of_mutation)
        population = new_population(individuals,mutated,elite_size)
    rank = fitness_rank(population, data)
    best_in_population.append([rank[0][1], population[rank[0][0]]])
    print(sorted(best_in_population)[0])

Przykładowe wywołanie algorytmu:

In [ ]:
if __name__ == "__main__":
    data48 = pd.read_excel('Dane_TSP_48.xlsx', engine='openpyxl')
    data76 = pd.read_excel('Dane_TSP_76.xlsx', engine='openpyxl')
    data127 = pd.read_excel('Dane_TSP_127.xlsx', engine='openpyxl')
    data = change_index(data127).values.tolist()
    print(genetic_algorithm(data, 400, 2000, 'roulette', 100, 'fittest', 'single_point', 0.005, 20))


# Algorytm NN

In [29]:
from tracemalloc import start
import pandas as pd
import xlrd

In [30]:
df48 = pd.read_excel("Dane_TSP_48.xlsx", index_col=0)
df76 = pd.read_excel("Dane_TSP_76.xlsx", index_col=0)
df127 = pd.read_excel("Dane_TSP_127.xlsx", index_col=0)

In [31]:
def sasiad(startowem,df):
    droga=0
    kolejnem=0
    trasa=[startowem]
    for j in range(len(df)-1):
        najkrotsza=1000000
        for i in range(len(df)):
            bylomiasto=False
            for k in range(len(trasa)):
                if(i+1==trasa[k]):
                    bylomiasto=True
                    break
            if(bylomiasto):
                pass
            elif(df[startowem].iloc[i]<najkrotsza and df[startowem].iloc[i]!=0):
                najkrotsza=df[startowem].iloc[i]
                kolejnem=i+1
        droga=droga+najkrotsza
        startowem=kolejnem
        trasa.append(startowem)
    print(droga)
    print(trasa)

Przykładowe wywołanie algorytmu:

In [ ]:
sasiad(1,df48)

In [33]:
def sasiadnajkrotsza(df):

    najkrotszadroga=1000000
    najkrotszatrasa=[]

    for l in range(len(df)):
        droga=0
        kolejnem=0
        trasa=[]
        startowem=l+1
        trasa.append(startowem)
        for j in range(len(df)-1):
            najkrotsza=1000000
            for i in range(len(df)):
                bylomiasto=False
                for k in range(len(trasa)):
                    if(i+1==trasa[k]):
                        bylomiasto=True
                        break
                if(bylomiasto):
                    pass
                elif(df[startowem].iloc[i]<najkrotsza and df[startowem].iloc[i]!=0):
                    najkrotsza=df[startowem].iloc[i]
                    kolejnem=i+1
            droga=droga+najkrotsza
            startowem=kolejnem
            trasa.append(startowem)
        if(droga<najkrotszadroga):
            najkrotszadroga = droga
            najkrotszatrasa = trasa
    print(najkrotszadroga)
    print(najkrotszatrasa)

Przykładowe wywołanie algorytmu:

In [ ]:
sasiadnajkrotsza(df127)